In [51]:
#Marnt Fendl Code

#Imports der benötigten Bibliotheken
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [52]:
#Pfade der Inputs festlegen
partnerlist = r'Partnerlisten\Top2000AS.xlsx'
keywordslist = 'Keywords/CradlepointKeywords.xlsx'

In [ ]:
#import Excel Partnerliste to partner_list array

df = pd.read_excel (partnerlist, header = None)
partner_list = df[0].tolist()
partner_list = [each_string.lower() for each_string in partner_list]

# initializing special characters
special_char = '@_!#$%^&*()<>?/\|}{~:;.[]'
 
# remove special characters 
partner_list = [''.join(filter(lambda i: i not in special_char, string)) for string in partner_list]

print(len(partner_list))
print(partner_list)

In [ ]:
#Bearbeitung der Sourcecodesnamen

#Array mit Sourcecodesnamen erstellen
sourcecodes_list = os.listdir('Sourcecodes')
#.txt Endung von Namen entfernen
sourcecodes_list = {x.replace('.txt', '') for x in sourcecodes_list}
#Ausgabe der Liste und Länge der Liste
print(len(sourcecodes_list))
print(sourcecodes_list)

In [ ]:
#Check wieviele der gewollte Sourcecodes sind schon runtergeladen
counter = 0
for partner in partner_list:
    if partner in sourcecodes_list:
        counter +=1

print(f'{counter} {len(partner_list)}')

In [ ]:
#Runterladen der Sourcecodes, welche noch nicht gefunden wurden 

#Festlegen von Arrays für die spätere Überprüfung der Partner
found = []
not_found = []

#Loop durch alle Partner in der Partnerliste
for index,company in enumerate(partner_list):
    #Wenn Sourcecode noch nicht runtergeladen, wird der Sroucecode runtergeladen
    if company not in sourcecodes_list:
        #Companyname Space mit Plus ersetzten, damit googlen möglich
        company.replace(' ', '+')
        #Leider hat die es mit Initalisierung außerhalb nicht geklappt
        driver = webdriver.Chrome()
        google = 'https://www.google.com/search?q='
        url = google+company
        #Aufrufen der Googlesuche
        driver.get(url)
        print(url)
        #Array mit allen gefundenen Links der Googlesuche erstellen
        results = driver.find_elements_by_partial_link_text('http')
        if results:
            print(results[0])
            #ersten Link aufrufen
            richtiger_link = results[0].get_attribute('href')
            print(richtiger_link)
            try:
                driver.get(richtiger_link)
                driver.implicitly_wait(2)
                company = company.replace('+', '')
                filename = f'{sourcecodes}\\{company}.txt'
                with open(filename,'w', encoding='utf-8') as f:
                    datafile = f.write(driver.page_source.lower())
                    f.close()
                driver.close()
                print(f'{index+1} von {len(partner_list)} fertig')
                found.append(company)
            except:
                print('Konnte Scourecodes nicht downloaden')
                not_found.append(company)
        else:
            print(f'Es wurde kein Link für {company} gefunden')
            not_found.append(company)
            
    else:
        next
        print(index)
        found.append(company)

In [61]:
#Wieviele Sourcecodes wurden gedownloaded?
print(f'Zu {len(found)} Partnern wurden Sourcecodes gedownloaded')
print(f'Zu {len(not_found)} Partnern wurden keine Sourcecodes gedownloaded')

Zu 1959 Partnern wurden Sourcecodes gedownloaded
Zu 41 Partnern wurden keine Sourcecodes gedownloaded


In [62]:
#Erstellen einer Funktion, welche den Sourcecode nach Keywords durchsucht und aus Findings ein Dictionary mit Keywords
#und Partnernamen erstellt
#hier kann man auch noch einen Counter einbauen, wie oft das Wort gefunden wurde
def check (filename, keywords):
    my_dict = {}
    with open(filename, encoding='utf-8') as f:
        datafile = f.readlines()
    for word in keywords:
        my_dict[word] = False
        for line in datafile:
            if word in line:
                my_dict[word] = True
    return my_dict

In [65]:
#Excel Keywords in Array 'Keywords' importieren
df = pd.read_excel (keywordslist, header = None)
keywords = []
keywords = df[0].tolist()
keywords = [str(i) for i in keywords]

print(keywords)

['cradlepoint', '5g', 'wireless wan', 'sd-wan', 'sd wan', 'wan', 'wired wan', 'wan service', '4g', 'lte', '5g router', 'router', 'High-Band 5G', '99999', '99.99', 'downtime', 'meraki', 'cisco', 'fortinet', 'digi', 'cellular networking', 'cellular network', 'networking', 'netzwerk', 'vernetzung', 'networks', 'network']


In [ ]:
#Erneut Sourcecodes in Array importieren für spätere Erstellung des gesamten Dictionarys
#Damit man den Code von oben nach unten ausführen kann
sourcecodes_list = os.listdir('Sourcecodes')
print(len(sourcecodes_list))
print(sourcecodes_list)

In [ ]:
#Die Check Funktion auf alle Partner der Partnerliste ausführen und aus den einzelen Dictionarys ein großes Dictionary für jeden
#Partner erstellen

complete_dict = {}
not_analyzed = []

for current_company in partner_list:
    try:
        #current_company = current_company.replace(' ','+')
        company_path = f'Sourcecodes\\{current_company}.txt'
        current_dict = check(company_path, keywords)
        complete_dict[current_company] = current_dict
    except:
        print(f'{current_company} hat nicht funktioniert')
        not_analyzed.append(current_company)

In [ ]:
#Dictionary in ein Pandas Dataframe umwandeln, damit man dieses als Excel exportieren kann 
data = pd.DataFrame.from_dict(complete_dict, orient='index')
print(data)
data.to_excel(f'Auswertungen/Top2000AS.xlsx') #Namen ändern für neue Datei